In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

import pandas as pd 

# importing functions
import sys
sys.path.append('/Users/audreyburggraf/Desktop/THESIS/Functions')

from functions import *
from unit_conversion_functions import *
from fitting_functions import *

In [2]:
df = pd.read_csv('/Users/audreyburggraf/Desktop/THESIS/Gaia Data Files/gaia_data.csv')

#  add the absolute gaia K band magnitude and stellar mass
df["M_ks"] = df.ks_m + 5 + 5*np.log10(df.parallax/1000)

# cut off certain range 
df = df[(4 < df.M_ks) & (df.M_ks < 11)].reset_index(drop=True)

# set the stellar mass 
df["stellar_mass"] =  find_star_mass(df.M_ks)

In [3]:
N1 = 70
N2 = 70000

seed = 8

np_parameters, parameters, prop_ra_synthetic, prop_dec_synthetic, parallax_ra_synthetic, parallax_dec_synthetic, planetary_ra_synthetic, planetary_dec_synthetic, prop_ra_model, prop_dec_model, parallax_ra_model, parallax_dec_model, planetary_ra_model, planetary_dec_model, noise_ra, noise_dec, errors, times_synthetic, times_model = find_signal_components(df, N1, N2, print_params='both')

N for synthetic data:  70
N for model data:  70000
 
Planetary parameters:
e: 0.11607003530679166  [unitless]
omega: 3.5759668607870405  [radians]
Omega: 1.072807089658365  [radians]
cos_i: 0.6376434203665425  [unitless]
m_planet: 8.702935835841917  [Jupiter masses]
P_orb: 0.17293455510715125  [years]
t_peri: 0.08568328606447792  [years]
 
Gaia parameters:
alpha0: 1  [degrees]
delta0: 2  [degrees]
mu_alpha: 115.32790772892858  [mas/year]
mu_delta: -355.3659662031838  [mas/year]
parallax: 50  [mas]
m_star: 0.26311240169941863  [M_sun]
x =  230965
 
astrometric_signature( [8.70293584] [0.2631124] [0.17293456] [20.] ) =  [314.01209372] [uas]


In [4]:
signal_ra_obs  = prop_ra_synthetic + parallax_ra_synthetic + planetary_ra_synthetic + noise_ra
signal_dec_obs = prop_dec_synthetic + parallax_dec_synthetic + planetary_dec_synthetic + noise_dec

In [6]:
# N1 = 5
# # 
# M = np.zeros((2*N1, 5), dtype=object)

# times_test = ['t1', 't2']
# parallax_ra = ['PI_ra(t1)', 'PI_ra(t2)']
# parallax_dec = ['PI_dec(t1)', 'PI_dec(t2)']

# for i in range(N1):
#     M[i,0] = 1 
#     M[i,1] = 0 
#     M[i,2] = times_test[i]
#     M[i,3] = 0
#     M[i,4] = parallax_ra[i]
    
#     M[i + N1, 0] = 0
#     M[i + N1, 1] = 1
#     M[i + N1, 2] = 0
#     M[i + N1, 3] = times_test[i]
#     M[i + N1, 4] = parallax_dec[i]

# for row in M:
#     print(row)

In [19]:
# ------------------------------- N O - P L A N E T - F I T --------------------------------------
def no_planet_fit(pars, signal_ra_obs, signal_dec_obs, errors, N, times): 
    alpha0, delta0, mu_alpha, mu_delta, parallax = pars
    
    param_nim = 3

    M = np.zeros((2*N, param_nim))

    parallax_ra, parallax_dec = generate_parallax_signal(alpha0, delta0, parallax, times) # [uas]
    

    for i in range(N):
        M[i,0] = times[i]
        M[i,1] = 0.0
        M[i,2] = parallax_ra[i]
        
        M[i+N, 0] = 0.0
        M[i+N, 1] = times[i]
        M[i+N, 2] = parallax_dec[i]
        
#     for row in M:
#         formatted_row = ['{:.0f}'.format(cell) for cell in row]
#         print(formatted_row)

    # finding the best fit values for the S samples
    signal_obs = np.concatenate((signal_ra_obs, signal_dec_obs))
    
    errors_long = np.concatenate((errors, errors))
    
    np_best_fit_val, _, _, _ = lstsq(M, signal_obs)
    

    # create empty arrays 
    array = np.zeros((2*N, param_nim))

    for i in range(N):
        x = np_best_fit_val 

        for j in range(param_nim):
            array[i,j] = M[i,j]*x[j]

    array_row_sums = np.sum(array, axis=1) 

    np_chi_sq = np.sum((array_row_sums - signal_obs)**2/errors_long**2)

    return np_best_fit_val, np_chi_sq
    # ----------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------


In [22]:
np_best_fit_val, np_chi_sq = no_planet_fit(np_parameters, signal_ra_obs, signal_dec_obs,errors,N1, times_synthetic)

alpha0, delta0, mu_alpha, mu_delta, parallax = np_parameters

np_fitted_params = mu_alpha, mu_delta, parallax

np_parameter_names = ['mu_alpha', 'mu_delta', 'parallax']

# for name, real_value, fitted_value in zip(parameter_names, parameters, fitted_params_1P):
#     difference = real_value - fitted_value
#     print(f'{name}: real={real_value}, fitted={fitted_value}, difference={np.abs(difference)}')


np_best_fit_val[0] *=1/1000
np_best_fit_val[1] *=1/1000

print(" ")


for name, real_value, fitted_value in zip(np_parameter_names, np_fitted_params, np_best_fit_val):
    difference = real_value - fitted_value
    print(f'{name: <10}: real={real_value.item():.5f}, fitted={fitted_value.item():.5f}, difference={np.abs(difference.item()):.5f}')
    

 
mu_alpha  : real=115.32791, fitted=115.34194, difference=0.01404
mu_delta  : real=-355.36597, fitted=-355.36189, difference=0.00408
parallax  : real=50.00000, fitted=1.00010, difference=48.99990


In [14]:


fitted_params_1P, wp_chi_sq_ra, wp_chi_sq_dec = one_planet_fit(parameters, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)


parameter_names = ['alpha0', 'delta0', 'mu_alpha', 'mu_delta', 'parallax', 'm_star', 'e', 'omega', 'Omega', 'cos_i', 'm_planet', 'P_orb', 't_peri']

# for name, real_value, fitted_value in zip(parameter_names, parameters, fitted_params_1P):
#     difference = real_value - fitted_value
#     print(f'{name}: real={real_value}, fitted={fitted_value}, difference={np.abs(difference)}')


for name, real_value, fitted_value in zip(parameter_names, parameters, fitted_params_1P):
    difference = real_value - fitted_value
    print(f'{name: <10}: real={real_value.item():.5f}, fitted={fitted_value.item():.5f}, difference={np.abs(difference.item()):.5f}')

alpha0    : real=1.00000, fitted=1.00010, difference=0.00010
delta0    : real=2.00000, fitted=1.99937, difference=0.00063
mu_alpha  : real=115.32791, fitted=115.32796, difference=0.00005
mu_delta  : real=-355.36597, fitted=-355.36695, difference=0.00099
parallax  : real=50.00000, fitted=49.99708, difference=0.00292
m_star    : real=0.26311, fitted=0.22764, difference=0.03547
e         : real=0.11607, fitted=0.11227, difference=0.00380
omega     : real=3.57597, fitted=3.53462, difference=0.04135
Omega     : real=1.07281, fitted=1.06121, difference=0.01159
cos_i     : real=0.63764, fitted=0.63899, difference=0.00135
m_planet  : real=8.70294, fitted=7.92285, difference=0.78008
P_orb     : real=0.17293, fitted=0.17292, difference=0.00002
t_peri    : real=0.08568, fitted=0.08447, difference=0.00122


In [23]:
wp_chi_sq_dec

65.43539723433533

In [24]:
wp_chi_sq_ra

66.3816530891677

In [15]:
Delta_BIC_ra = BIC(np_chi_sq, wp_chi_sq_dec, N1)
Delta_BIC_ra

-25120508802.30738

In [16]:
Delta_BIC_dec = BIC(np_chi_sq, wp_chi_sq_dec, N1)
Delta_BIC_dec

-25120508802.30738